Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

In [93]:
import pandas as pd
from sklearn.utils import resample 

train = pd.read_csv('C:\\Users\\becca\\Desktop\\DS-Unit-4-Sprint-1-NLP\\module3-document-classification\\train.csv')
test = pd.read_csv('C:\\Users\\becca\\Desktop\\DS-Unit-4-Sprint-1-NLP\\module3-document-classification\\test.csv')

minority = train[train['ratingCategory'] == 0]
majority = train[train['ratingCategory'] == 1]

df_minority_upsampled = resample(minority, replace=True, n_samples=majority.shape[0])

df_upsampled = pd.concat([majority, df_minority_upsampled])

In [94]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [95]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [96]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,3))

clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

In [97]:
#define search space
parameters = {
    'vect__max_df' : (0.75, 1.0),
    'clf__max_depth': (5,10,15,20)
}

In [98]:
grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4,verbose=1)

In [99]:
grid_search.fit(df_upsampled['description'], df_upsampled['ratingCategory'])

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  40 out of  40 | elapsed:    2.7s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 3),
                                                        no

In [100]:
grid_search.best_score_

0.9714285714285715

In [101]:
from sklearn.metrics import accuracy_score

y_pred = grid_search.predict(train['description'])

acc = accuracy_score(train['ratingCategory'], y_pred)
print(acc)

0.01639344262295082


In [102]:
#make a submission file
#predictions on test sample
pred = grid_search.predict(test['description'])
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')


In [89]:
#make sure the category is an integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [103]:
#save submission file
submission.to_csv(f'.\\data\\submission{subNumber}.csv', index=False)
subNumber += 1